In [ ]:
#import the train dataset with volatility targets for each stock and time id
import pandas as pd
import numpy as np
import plotly.express as px
train = pd.read_csv('train.csv')
train.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [ ]:
#import the whole book data
import os
from sklearn.metrics import r2_score
import glob
list_order_book_file_train = glob.glob('book_train.parquet/*')

In [3]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

In [4]:

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

In [ ]:
#compute realized volatility for each time id
def realized_volatility_per_time_id(file_path, prediction_column_name):
    df_book_data = pd.read_parquet(file_path)
    df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (
                                      df_book_data['bid_size1']+ df_book_data[
                                  'ask_size1'])
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].transform(log_return)
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
    stock_id = file_path.split('=')[1]
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
    return df_realized_vol_per_stock[['row_id',prediction_column_name]]

In [ ]:
#compute realized volatility for 100 seconds intervals in each time id (600 seconds)
def realized_volatility_per_time_id_every_100_seconds(file_path, prediction_column_prefix):
    import pandas as pd
    import numpy as np

    df_book_data = pd.read_parquet(file_path)
    
    # Calcolo WAP
    df_book_data['wap'] = (
        df_book_data['bid_price1'] * df_book_data['ask_size1'] +
        df_book_data['ask_price1'] * df_book_data['bid_size1']
    ) / (df_book_data['bid_size1'] + df_book_data['ask_size1'])
    
    # Creiamo gli intervalli temporali
    df_book_data['interval'] = (df_book_data['seconds_in_bucket'] // 100) * 100  # 0, 100, ..., 500
    df_book_data = df_book_data[df_book_data['interval'] <= 500]

    results = []
    stock_id = file_path.split('=')[1]

    for (time_id, interval), group in df_book_data.groupby(['time_id', 'interval']):
        group = group.sort_values('seconds_in_bucket')
        group['log_return'] = log_return(group['wap'])
        group = group.dropna(subset=['log_return'])
        if not group.empty:
            vol = realized_volatility(group['log_return'])
            row_id = f'{stock_id}-{time_id}'
            results.append({
                'row_id': row_id,
                'interval': interval,
                'volatility': vol
            })

    df_results = pd.DataFrame(results)

    # Pivot in modo sicuro: riga = row_id, colonne = intervalli, valori = volatilità
    df_pivot = df_results.pivot_table(index='row_id', columns='interval', values='volatility')

    # Rinomina le colonne in modo chiaro
    df_pivot.columns = [f"{prediction_column_prefix}_{int(c)}" for c in df_pivot.columns]
    df_pivot = df_pivot.reset_index()

    return df_pivot



In [ ]:
#iterate for each stock
def past_realized_volatility_per_stock(list_file,prediction_column_prefix):
    df_past_realized = pd.DataFrame()
    
    for file in list_file:
            df_one_stock = realized_volatility_per_time_id(file, prediction_column_prefix)
            df_past_realized = pd.concat([df_past_realized, df_one_stock], axis=0, ignore_index=True)
    return df_past_realized
df_past_realized_train = past_realized_volatility_per_stock(list_file=list_order_book_file_train,
                                                           prediction_column_prefix='sigma')

In [ ]:
#iterate for each stock (100 seconds batches version)
def past_realized_volatility_per_stock_every_100_seconds(list_file,prediction_column_prefix):
    df_past_realized = pd.DataFrame()
    
    for file in list_file:
            df_one_stock = realized_volatility_per_time_id_every_100_seconds(file, prediction_column_prefix)
            df_past_realized = pd.concat([df_past_realized, df_one_stock], axis=0, ignore_index=True)
    return df_past_realized

#run 20 minutes
df_past_realized_train_every_100_seconds = past_realized_volatility_per_stock_every_100_seconds(list_file=list_order_book_file_train,
                                                           prediction_column_prefix='sigma')

In [ ]:
#create a dataset that contains a row id (stock id-time id) and all the realized volatilities (100 seconds batches)
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id', 'target']]

volatility_columns = ['row_id'] + [col for col in df_past_realized_train_every_100_seconds.columns if col.startswith('sigma')]

df_joined_every_100_seconds = train.merge(df_past_realized_train_every_100_seconds[volatility_columns], on='row_id', how='left')


In [10]:
train = pd.read_csv('train.csv')

In [ ]:
#create a dataset that contains a row id (stock id-time id) and the realized volatility
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id', 'target']]

volatility_columns = ['row_id'] + [col for col in df_past_realized_train.columns if col.startswith('sigma')]

df_joined = train.merge(df_past_realized_train[volatility_columns], on='row_id', how='left')

In [ ]:
#sigma_0 is the realized volatility between second 0 and 100, sigma_100 between 100 and 200, and so on
print(df_joined)
print(df_joined_every_100_seconds)

           row_id    target     sigma
0             0-5  0.004136  0.004499
1            0-11  0.001445  0.001204
2            0-16  0.002168  0.002369
3            0-31  0.002195  0.002574
4            0-62  0.001747  0.001894
...           ...       ...       ...
428927  126-32751  0.003461  0.003691
428928  126-32753  0.003113  0.004104
428929  126-32758  0.004070  0.003118
428930  126-32763  0.003357  0.003661
428931  126-32767  0.002090  0.002091

[428932 rows x 3 columns]
           row_id    target   sigma_0  sigma_100  sigma_200  sigma_300  \
0             0-5  0.004136  0.001978   0.002181   0.001689   0.001816   
1            0-11  0.001445  0.000387   0.000521   0.000218   0.000298   
2            0-16  0.002168  0.000864   0.001049   0.001398   0.000535   
3            0-31  0.002195  0.000336   0.001725   0.000535   0.001466   
4            0-62  0.001747  0.000987   0.000383   0.000393   0.000498   
...           ...       ...       ...        ...        ...        ...   